# Fantasy Football Analysis - Fixed Version

This notebook loads your CSV data and applies your league's custom scoring system.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries loaded successfully!")

In [ ]:
# Load the CSV file - skip the header rows to find the actual data
import os
os.chdir('/Users/ben/projects/fantasy-football-draft-spreadsheet')

# Try to read the CSV and inspect its structure
try:
    # Read first few lines to understand structure
    with open('CSG Fantasy Football Sheet - 2025 v13.01.csv', 'r') as f:
        lines = f.readlines()[:10]
    
    print("First 10 lines of CSV:")
    for i, line in enumerate(lines):
        print(f"Line {i}: {line[:100]}...")
        
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
# Find the actual header row with player data
# Look for row that starts with "Rank" or has player names

def find_header_row(filename, max_rows=20):
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            if i > max_rows:
                break
            # Look for common fantasy football column headers
            line_lower = line.lower()
            if any(word in line_lower for word in ['rank', 'player', 'position', 'team', 'owner']):
                print(f"Potential header found at line {i}: {line[:150]}")
                return i
    return None

header_row = find_header_row('CSG Fantasy Football Sheet - 2025 v13.01.csv')
print(f"\nHeader row appears to be at line: {header_row}")

In [ ]:
# Load the data using the correct header row
if header_row is not None:
    df = pd.read_csv('CSG Fantasy Football Sheet - 2025 v13.01.csv', 
                     skiprows=header_row, 
                     low_memory=False)
    
    print(f"Loaded dataframe with shape: {df.shape}")
    print(f"\nColumn names:")
    for i, col in enumerate(df.columns[:20]):  # Show first 20 columns
        print(f"{i}: {col}")
    
    print(f"\nFirst few rows:")
    display(df.head(3))
else:
    print("Could not find header row. Let's try a different approach.")

In [ ]:
# Load league configuration
try:
    with open('config/league-config.yaml', 'r') as f:
        config = yaml.safe_load(f)
    
    print("League Configuration Loaded:")
    print(f"League Name: {config['league_name']}")
    print(f"Teams: {config['basic_settings']['teams']}")
    print(f"Roster slots: {config['roster']['roster_slots']}")
    
    print("\nScoring System:")
    print(f"Passing yards: {config['scoring']['passing']['yards']} per yard")
    print(f"Rushing yards: {config['scoring']['rushing']['yards']} per yard")
    print(f"Receiving yards: {config['scoring']['receiving']['yards']} per yard")
    
except Exception as e:
    print(f"Error loading config: {e}")

In [ ]:
# Clean and process the data
if 'df' in locals() and df is not None:
    # Remove completely empty rows
    df = df.dropna(how='all')
    
    # Try to identify key columns
    print("Looking for key fantasy columns...")
    
    # Look for player name column
    player_cols = [col for col in df.columns if any(word in col.lower() for word in ['player', 'name'])]
    position_cols = [col for col in df.columns if any(word in col.lower() for word in ['position', 'pos'])]
    team_cols = [col for col in df.columns if any(word in col.lower() for word in ['team'])]
    
    print(f"Potential player columns: {player_cols}")
    print(f"Potential position columns: {position_cols}")
    print(f"Potential team columns: {team_cols}")
    
    # Show some sample data
    if len(df) > 0:
        print(f"\nSample data (first 5 rows, first 10 columns):")
        display(df.iloc[:5, :10])
else:
    print("No data loaded yet.")

In [ ]:
# Manual column mapping if automatic detection doesn't work
# Let's manually examine the columns and map them

if 'df' in locals() and len(df) > 0:
    print("Let's examine the actual data structure:")
    
    # Show each column with some sample values
    for i, col in enumerate(df.columns[:15]):
        sample_values = df[col].dropna().head(3).tolist()
        print(f"Column {i} '{col}': {sample_values}")
    
    print("\n" + "="*50)
    print("Based on the data, let's manually map the important columns...")

## Quick Analysis

Once we have the data properly loaded, we can do some basic analysis:

In [ ]:
# Basic analysis function
def analyze_players(df, player_col, position_col):
    """
    Simple analysis of player data
    """
    if player_col in df.columns and position_col in df.columns:
        # Count by position
        position_counts = df[position_col].value_counts()
        print("Players by position:")
        print(position_counts)
        
        # Show top 10 players
        print("\nTop 10 players:")
        display(df[[player_col, position_col]].head(10))
        
        return True
    else:
        print(f"Columns not found: {player_col}, {position_col}")
        return False

# This will be filled in once we identify the correct columns
print("Analysis function ready. Will run once columns are identified.")

In [ ]:
# Value-Based Drafting calculation
def calculate_vbd(df, points_col, position_col, replacement_levels):
    """
    Calculate Value Based Drafting scores
    """
    if points_col not in df.columns or position_col not in df.columns:
        print(f"Required columns not found: {points_col}, {position_col}")
        return df
    
    df_copy = df.copy()
    df_copy['VBD'] = 0
    
    for pos, replacement_rank in replacement_levels.items():
        if pos == 'FLEX':  # Skip FLEX for now
            continue
            
        pos_players = df_copy[df_copy[position_col] == pos].copy()
        if len(pos_players) >= replacement_rank:
            pos_players = pos_players.sort_values(points_col, ascending=False)
            replacement_points = pos_players.iloc[replacement_rank-1][points_col]
            
            # Calculate VBD for this position
            pos_mask = df_copy[position_col] == pos
            df_copy.loc[pos_mask, 'VBD'] = df_copy.loc[pos_mask, points_col] - replacement_points
    
    return df_copy

print("VBD calculation function ready.")

## Next Steps

Run the cells above to:
1. Load and inspect your CSV data
2. Identify the correct column mappings
3. Apply your league's scoring system
4. Calculate Value-Based Drafting rankings

Once we have the data properly loaded, we can add more advanced analysis and visualizations!